In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model
model = create_model()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
#użycie callbacks w punktach kontrolnych
checkpoint_path = 'training_1/cp.ckpt.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [8]:
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
28/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4779 - loss: 1.6983
Epoch 1: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5040 - loss: 1.6224 - val_accuracy: 0.7760 - val_loss: 0.7122
Epoch 2/10
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8556 - loss: 0.4549
Epoch 2: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8566 - loss: 0.4531 - val_accuracy: 0.8450 - val_loss: 0.5168
Epoch 3/10
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9084 - loss: 0.3106
Epoch 3: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9094 - loss: 0.3094 - val_accuracy: 0.8570 - val_loss: 0.4805
Epoch 4/10
27/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9647 - loss: 0.2047
Epoch 4: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9626 - loss: 0.2057 - val_accuracy: 0.8470

In [9]:
os.listdir(checkpoint_dir)

['cp.ckpt.weights.h5']

In [10]:
model=create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"niewytrenowany model-> acc:{acc}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 - 0s - 10ms/step - accuracy: 0.1380 - loss: 2.2687
niewytrenowany model-> acc:0.1379999965429306


In [11]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"model z wagami-> acc:{acc}")

32/32 - 0s - 3ms/step - accuracy: 0.8730 - loss: 0.3985


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


model z wagami-> acc:0.8730000257492065


In [12]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size=32

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=5*batch_size)

In [16]:
import os

os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback],
          batch_size=batch_size,
          verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 5: saving model to training_2/cp-0005.ckpt.weights.h5

Epoch 10: saving model to training_2/cp-0010.ckpt.weights.h5

Epoch 15: saving model to training_2/cp-0015.ckpt.weights.h5

Epoch 20: saving model to training_2/cp-0020.ckpt.weights.h5

Epoch 25: saving model to training_2/cp-0025.ckpt.weights.h5

Epoch 30: saving model to training_2/cp-0030.ckpt.weights.h5

Epoch 35: saving model to training_2/cp-0035.ckpt.weights.h5

Epoch 40: saving model to training_2/cp-0040.ckpt.weights.h5

Epoch 45: saving model to training_2/cp-0045.ckpt.weights.h5

Epoch 50: saving model to training_2/cp-0050.ckpt.weights.h5


In [17]:
os.listdir(checkpoint_dir)

['cp-0025.ckpt.weights.h5',
 'cp-0040.ckpt.weights.h5',
 'cp-0035.ckpt.weights.h5',
 'cp-0030.ckpt.weights.h5',
 'cp-0000.ckpt.weights.h5',
 'cp-0015.ckpt.weights.h5',
 'cp-0020.ckpt.weights.h5',
 'cp-0050.ckpt.weights.h5',
 'cp-0005.ckpt.weights.h5',
 'cp-0010.ckpt.weights.h5',
 'cp-0045.ckpt.weights.h5']

In [20]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
if latest is not None:
    model = create_model()
    model.load_weights(latest)
    loss, acc = model.evaluate(test_images, test_labels, verbose=2)
    print(f"model latest-> acc:{acc}")
else:
    print("No checkpoint files found.")

No checkpoint files found.


In [23]:
model = create_model()
model.load_weights('training_2/cp-0050.ckpt.weights.h5')

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"model latest-> acc:{acc}")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 11ms/step - accuracy: 0.8840 - loss: 0.4832
model latest-> acc:0.8840000033378601


In [25]:
#zapis całego modelu
model = create_model()
model.fit(train_images,train_labels,epochs=5)

!mkdir -p saved_model
model.save('saved_model/mojmod.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4858 - loss: 1.6098
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8742 - loss: 0.4517
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9249 - loss: 0.2892
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9550 - loss: 0.2170
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9701 - loss: 0.1541


In [26]:
new_model = tf.keras.models.load_model('saved_model/mojmod.h5')
new_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                     │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)
model.save('mm.h5')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.1384 - accuracy: 0.6770
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4086 - accuracy: 0.8930
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2771 - accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2033 - accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1535 - accuracy: 0.9650


In [ ]:
next_model = tf.keras.models.load_model('mm.h5')
next_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
